---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-10-20 12:55:33--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.22.33, 188.185.33.206, 188.185.10.78, ...
Connecting to zenodo.org (zenodo.org)|188.185.22.33|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2023-10-20 12:55:34--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  33.9MB/s    in 47s     

2023-10-20 12:56:21 (29.4 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# Получаем топ-5 близких слов к "dog"
similar_words = wv_embeddings.most_similar('dog', topn=5)

# Проверяем, входит ли "cat" в этот список и находим его место
cat_rank = None
for rank, (word, score) in enumerate(similar_words, 1):
    if word == 'cat':
        cat_rank = rank
        break

if cat_rank:
    print(f"Слово 'cat' входит в топ-5 близких слов к 'dog' и занимает {cat_rank}-е место.")
else:
    print("Слово 'cat' не входит в топ-5 близких слов к 'dog'.")


Слово 'cat' не входит в топ-5 близких слов к 'dog'.


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        self.tokenizer = WordPunctTokenizer()

    def tokenize(self, text):
        return self.tokenizer.tokenize(text)


In [ ]:
import numpy as np

def question_to_vec(question, embeddings, tokenizer, dim=200):
    words = tokenizer.tokenize(question)  # Токенизация вопроса
    vec = np.zeros(dim)
    count = 0

    for word in words:
        if word in embeddings:
            vec += embeddings[word]
            count += 1

    if count == 0:
        return vec

    return vec / count

# Пример использования:
question = "How to use this model for text classification?"
tokenizer = WordPunctTokenizer()
vec = question_to_vec(question, wv_embeddings, tokenizer, dim=200)
vec

array([-0.02997803,  0.01074622, -0.02543433,  2.10495943, -2.77936167,
        0.94543868,  2.87577301,  0.35719535, -0.0226183 , -0.95616852,
       -0.41931471, -0.52559266, -0.8271836 ,  0.1341154 ,  0.76798845,
        3.17659128,  0.64646575, -0.62409683, -0.20443121, -0.50592439,
       -0.23403781, -0.93461779,  1.42134985,  1.55444178, -0.46888775,
        1.0073745 , -0.96590012,  1.81124368,  0.44505042, -1.56509948,
       -0.35809117,  0.62071769,  0.97005779, -0.58053043, -1.32712597,
        0.96752301, -1.56819451,  0.8458265 ,  0.65999672,  0.16630841,
       -0.53354374, -0.57650164, -0.02148117,  0.27221009,  0.86965055,
       -1.28568958, -0.81224487,  1.52408952,  1.25316628, -0.60878244,
        2.36439544, -1.36095566, -0.58411634, -0.30012681, -1.2512877 ,
        0.73342204,  0.02393602,  2.30025315, -1.1112442 ,  0.04553366,
        0.29995277, -0.82065198, -0.3095444 , -0.96164137, -0.1010743 ,
       -1.7667941 ,  0.31055668, -0.80691531, -0.31953064, -0.22

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
sentence = "I love neural networks"
vec = question_to_vec(sentence, wv_embeddings, tokenizer, dim=200)

# Теперь получим третью компоненту (с индексом 2) и округлим до 2 знаков после запятой
third_component = round(vec[2], 2)

print(f"Третяя компонента вектора предложения: {third_component}")


Третяя компонента вектора предложения: -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

In [ ]:
rank_qi = 2 # Ранг дубликата qi
K_val = [1, 47] # Значение K

Hits = [1 if rank_qi <= K else 0 for K in K_val]

DCG = [1 / (np.log2(1 + rank_qi)) if rank_qi <= K else 0 for K in K_val]

for i, K in enumerate(K_val):
    print("Hits@",K,Hits[i])
    print("DCG@",K,DCG[i])
print("Hits@47 - DCG@1 =",Hits[1] - DCG[0])

Hits@ 1 0
DCG@ 1 0
Hits@ 47 1
DCG@ 47 0.6309297535714575
Hits@47 - DCG@1 = 1


#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
import math

def hits_count(dup_ranks, k):
    hits_value = sum([1 for rank in dup_ranks if rank <= k])/len(dup_ranks)
    return hits_value

def dcg_score(dup_ranks, k):
    dcg_value = sum([1 / math.log2(1 + rank) for rank in dup_ranks if rank <= k])/len(dup_ranks)
    return dcg_value

# Пример использования:
dup_ranks = [9]  # rank_q′i=9
k = 10
dcg_10 = dcg_score(dup_ranks, k)
print(f"DCG@{k} = {dcg_10:.1f}")


DCG@10 = 0.3


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split("\t"))
    return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('/content/drive/MyDrive/datasets/stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    # Преобразуем вопрос в вектор
    question_vector = question_to_vec(question, embeddings, tokenizer, dim)

    # Создаем список пар (позиция, кандидат) и сортируем его на основе косинусного расстояния
    ranked_candidates = [(i, candidate) for i, candidate in enumerate(candidates)]
    ranked_candidates.sort(key=lambda x: -cosine_similarity([question_vector], [question_to_vec(x[1], embeddings, tokenizer, dim)])[0, 0])

    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [('*', 'Getting all list items of an unordered list in PHP'), #скрыт
            ('*', 'select2 not displaying search results'), #скрыт
            ('*', 'WPF- How to update the changes in list item of a list')]] #скрыт
results

[[(1, 'C# create cookie from string and send it'),
  (0, 'Convert Google results object (pure js) to Python object'),
  (2, 'How to use jQuery AJAX for an outside domain?')],
 [('*', 'Getting all list items of an unordered list in PHP'),
  ('*', 'select2 not displaying search results'),
  ('*', 'WPF- How to update the changes in list item of a list')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('/content/drive/MyDrive/datasets/stackoverflow_similar_questions/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
len(train_data)

1000000

In [ ]:
train_data[:5]

[['converting string to list',
  'Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP\n'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it\n'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list\n']]

In [ ]:
words = [' '.join(line) for line in train_data]
mean_len = len(' '.join(words).split())/len(words)
mean_len

19.166228

In [ ]:
'''your code'''

'your code'

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=20).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.411 | Hits@   1: 0.411
DCG@   5: 0.420 | Hits@   5: 0.428
DCG@  10: 0.422 | Hits@  10: 0.434
DCG@ 100: 0.437 | Hits@ 100: 0.517
DCG@ 500: 0.477 | Hits@ 500: 0.840
DCG@1000: 0.494 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

### Эмбеддинги, обученные на корпусе похожих вопросов

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

## Использование FastText:

In [ ]:
from gensim.models import FastText

In [ ]:
embeddings_trained2 = FastText(words,
                 vector_size=200,
                 min_count=5,
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained2, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.064 | Hits@   1: 0.064
DCG@   5: 0.080 | Hits@   5: 0.095
DCG@  10: 0.086 | Hits@  10: 0.114
DCG@ 100: 0.117 | Hits@ 100: 0.277
DCG@ 500: 0.161 | Hits@ 500: 0.627
DCG@1000: 0.200 | Hits@1000: 1.000


In [ ]:
embeddings_trained2.most_similar("database", topn=5)

[('’', 0.19397179782390594),
 ('w', 0.15904733538627625),
 ('P', 0.1481848806142807),
 ('/', 0.1408521831035614),
 ('‘', 0.13730961084365845)]

### Использую предпроцессинг

In [ ]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object\n']

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
con = [' '.join(line) for line in train_data]
con[:10]

['converting string to list Convert Google results object (pure js) to Python object\n',
 'Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?\n',
 'Sending array via Ajax fails Getting all list items of an unordered list in PHP\n',
 'How to insert CookieCollection to CookieContainer? C# create cookie from string and send it\n',
 'Updating one element of a bound Observable collection WPF- How to update the changes in list item of a list\n',
 'MongoDB error on find() Retrieve only the queried element in an object array in MongoDB collection\n',
 'select2 not displaying search results How to use jQuery AJAX for an outside domain?\n',
 'Using Reduce to merge multiple data frames with passing arguments and without defining function outside the Reduce (syntax) R - merge a list of data frames into one data frame with missing values by row\n',
 'Adding Prototype to JavaScript Object Literal How does JavaScript .prototype wor

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def process_text(text):
    text = text.lower()
    text = re.sub(r'[\d]+|\n', '', text)
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word.isalpha()]

    return lemmatized_words

processed_data = [process_text(text) for text in con]

In [ ]:
processed_data[0]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(processed_data, # data for model to train on
                 vector_size=200,         # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
embeddings_trained.most_similar("database", topn=5)

## Использование FastText:

In [ ]:
embeddings_trained2 = FastText(processed_data,
                 vector_size=200,
                 min_count=5,
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained2, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
embeddings_trained2.most_similar("database", topn=5)

# Уберу обработку стоп-слов и снова обучу систему

In [ ]:
def process_text(text):
    text = text.lower()
    text = re.sub(r'[\d]+|\n', '', text)
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]

    return lemmatized_words

processed_data2 = [process_text(text) for text in con]

In [ ]:
processed_data2[0]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(processed_data2, # data for model to train on
                 vector_size=200,         # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
embeddings_trained.most_similar("database", topn=5)

## Вывод: DCG@1000 356 правильных, из Hits@1000 1000 правильных

## Использование FastText:

In [ ]:
embeddings_trained2 = FastText(processed_data2,
                 vector_size=200,
                 min_count=5,
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 3760
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained2, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [ ]:
embeddings_trained2.most_similar("database", topn=5)

## Вывод: DCG@1000 200 правильных, из Hits@1000 1000 правильных

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Word2Vec лучше работает на векторизированных данных со стоп словами, FastText показывает лучший результат без стоп слов
